### Trabalho Final
***
- 1606881 | Bruna Mirele Martins da Silveira
- 1606894 | Caio Wey Barros
- 1606966 | Gustavo Matheus Santos Penedo
***

#### Objetivos de Ensino

#### Exercitar os seguintes conceitos:
1. Coleta de dados
2. Analisar e explorar os conjuntos de dados
3. Analisar e realizar tratamento de dados se necessário
4. Realizar agrupamentos de dados
5. Analisar resultados obtidos
6. Responder as questões objetivas

#### Enunciado da Atividade Prática: Análise de Dados

Você foi contratado para criar uma automação de fluxo de dados completa para dados clínicos dos pacientes em um projeto de pesquisa de saúde. Seus dados incluem três conjuntos principais: dados_clinicos, dados_pacientes e dados_estados. Cada conjunto de dados possui as seguintes colunas:
1. **dados_clinicos**: contém informações sobre os pacientes como id_cliente, peso, colesterol e ganero.
2. **dados_pacientes**: fornece detalhes sobre os pacientes como id_cliente, idade, classe_trabalho, escolaridade, id_estado, estado_civil, raca, qtde_filhos e salario.
3. **dados_estados**: oferece informações sobre os estados, incluindo id_estado, sigla, estado, regiao e pais.

Os datasets utilizados no trabalho podem ser obtidos nesse [link](https://leandrolessa.com.br/wp-content/uploads/2024/06/dados_clinicos_pacientes.zip).

#### Tarefas:

In [ ]:
# Bibliotecas
import pandas as pd
import os
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
load_dotenv('../config/.env')

True

**1. Coleta de Dados**

Realize a coleta de dados dos pacientes disponibilizados.

In [ ]:
# Insira o código aqui

**2. Integração dos Dados**

Realize um inner join entre dados_clinicos, dados_pacientes e dados_estados para obter uma visão completa dos dados dos pacientes.

In [1]:
# Insira o código aqui

**3. Correção dos Dados**

Identifique e corrija dados inconsistentes, como valores ausentes, duplicados ou incorretos.

In [2]:
# Insira o código aqui

**4. Armazenamento de Dados**

Desenvolva o diagrama de entidade-relacionamento entre as tabelas e persista os dados no banco de dados MySQL.

In [ ]:
# Conexão com o banco de dados
def conectar_banco_mysql():
    try:
        user = os.environ['user_db']
        password = os.environ['password_db']
        host = os.environ['host']
        conexao = f'mysql://{user}:{password}@{host}'
        engine = create_engine(conexao)
        conn = engine.connect()
        print('Conexão realizada com sucesso!')
        return conn
    except Exception as e:
        print(f'Não foi possível fazer conexão com o banco de dados. ERRO: {e}')

# Validação da conexão com o servidor de banco de dados
conn = conectar_banco_mysql()

# Criação do banco de dados
try:
    database = 'pacientes'
    query = f'create schema if not exists {database}'
    conn.execute(text(query))
    print('Banco de dados criado com sucesso!')
    conn.commit()
except Exception as e:
    print(f'Não foi possível criar o banco de dados. ERRO: {e}')

# Execução do script de criação das tabelas e seus relacionamentos
with open('../script_DB/create_database_paciente.sql', 'r') as file:
    query = file.read()

    try:
        conn.execute(text(query))
        print('Tabelas criadas com sucesso!')
        conn.commit()
    except Exception as e:
        print(f'Não foi possível criar as tabelas. ERRO: {e}')

# Inserção de registros no banco de dados
def inserir_dados_tabelas(lista_dados, nome_tabela):
    coluna_tabela = nome_tabela[3:]

    for registro in lista_dados:
        try:
            query = f'''
                INSERT INTO {nome_tabela} ({coluna_tabela})
                VALUES ('{registro}')
            '''
            conn.execute(text(query))
            conn.commit()
            print(f'Registro inserido com sucesso: {registro}')
        except Exception as e:
            print(f'Não foi possível inserir o registro: {registro}. ERRO: {e}')
            conn.rollback()

Conexão realizada com sucesso!
Banco de dados criado com sucesso!
Tabelas criadas com sucesso!


**5. Consultas SQL**

Após o armazenamento dos dados, responda às perguntas do questionário utilizando SQL.

In [4]:
# Insira o código aqui

#### **ATENÇÃO PARA TRATAMENTO DE DADOS**

Avaliem se será necessário realizar tratamento de dados ausentes nos datasets disponibilizados.

**Instruções para correção de dados ausentes**

- Moda para Variáveis Categóricas: Preencher valores ausentes nas colunas categóricas com a moda.
- Mediana para Variáveis Numéricas: Preencher valores ausentes nas colunas numéricas com a mediana.

#### Utilize o diagrama de entidade e relacionamento

![Diagrama de entidade e relacionamento](../doc/Diagrama%20de%20Entidade%20e%20Relacionamento.png)

#### Dicas do professor:

1. Antes de finalizar suas respostas, certifique-se de verificar se o gabarito está corretamente preenchido.
2. Preste atenção aos requisitos específicos de cada questão para garantir uma resposta adequada.
3. Elimine dados duplicados.
4. Lembre-se de que os dados fornecidos nos datasets são fictícios e não refletem dados do mundo real.
5. Siga rigorosamente todas as etapas descritas no enunciado das questões para uma análise completa e precisa.

#### Perguntas do Trabalho

**1. Qual é a média e o desvio padrão do colesterol das pessoas que têm peso superior a 120kg e residem na região Sul do Brasil?**

In [ ]:
# Insira o código aqui

**2. Qual é a classe de trabalho com maior número de pessoas e qual é o número correspondente na base de dados?**

In [ ]:
# Insira o código aqui

**3. Qual é a classe de trabalho que apresenta a maior média de salários e qual é o valor associado para as pessoas do estado de Pernambuco?**

In [ ]:
# Insira o código aqui

**4. Qual estado civil possui a menor quantidade de filhos e qual é esse número?**

In [ ]:
# Insira o código aqui

**5. Quantas pessoas casadas possuem filhos e têm um salário acima de 3000?**

In [ ]:
# Insira o código aqui

**6. Qual é a média salarial das pessoas casadas que possuem ensino superior completo e trabalham como Fncionário do Setor Privado?**

In [ ]:
# Insira o código aqui

**7. Qual estado civil possui a menor quantidade de filhos e qual é esse número?**

In [ ]:
# Insira o código aqui

**8. Quantas pessoas casadas possuem filhos e têm um salário acima de 3000?**

In [ ]:
# Insira o código aqui

**9. Qual é a soma das idades de todas as mulheres solteiras que residem em Santa Catarina?**

In [ ]:
# Insira o código aqui

**10. Qual o estado que possui a maior média de peso?**

In [ ]:
# Insira o código aqui